In [1]:
# Установка
!pip -q install ultralytics shapely opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00


In [ ]:
# =========================
# TRAIN MULTI-GSD + ADAPTIVE CONF + 3-SCALE INFERENCE
# + TILE BATCHING + MEANS + CHECKPOINTS
# =========================

import os, glob, random
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from ultralytics import YOLO
import math

# -------------------------
# Google Drive mount
# -------------------------
from google.colab import drive
drive.mount('/content/drive')

# -------------------------
# Repro
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# =========================
# Unzip dataset from Google Drive
# =========================
ZIP_ON_DRIVE = "/content/drive/MyDrive/dataset.zip"
OUT_ZIP = "/content/dataset.zip"
OUT_DIR = "/content/dataset"

assert os.path.exists(ZIP_ON_DRIVE), f"Dataset zip not found on Drive: {ZIP_ON_DRIVE}"

!cp -v "{ZIP_ON_DRIVE}" "{OUT_ZIP}"
!mkdir -p "{OUT_DIR}"
!unzip -q "{OUT_ZIP}" -d "{OUT_DIR}"
print("dataset dir:", OUT_DIR)

IMG_DIR  = os.path.join(OUT_DIR, "images")
MASK_DIR = os.path.join(OUT_DIR, "gt")

assert os.path.isdir(IMG_DIR),  f"Missing images dir: {IMG_DIR}"
assert os.path.isdir(MASK_DIR), f"Missing gt dir: {MASK_DIR}"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
'/content/drive/MyDrive/dataset.zip' -> '/content/dataset.zip'


In [8]:
# -------------------------
# Split function
# -------------------------
def split_by_scene(imgs, val_ratio=0.15, seed=42):
    rng = random.Random(seed)
    idxs = list(range(len(imgs)))
    rng.shuffle(idxs)
    n_val = max(1, int(len(imgs) * val_ratio))
    val = set(idxs[:n_val])

    train_imgs = [p for i, p in enumerate(imgs) if i not in val]
    val_imgs   = [p for i, p in enumerate(imgs) if i in val]
    return train_imgs, val_imgs

# =========================
# Detection settings
# =========================
MODEL_WEIGHTS = "yolo26x-obb.pt"
CAR_CLASS_ID = 10

TILE_SIZE = 1024
OVERLAP = 0.1

# adaptive thresholds
CONF_PRIMARY  = 0.10
CONF_FALLBACK = 0.01  # <-- fallback: более низкий порог

IOU_THRES_TILE = 0.5

DEVICE = None
HALF = True

MERGE_CELL_PX = 3

# -------------------------
# Tile batching
# -------------------------
TILE_BATCH = 16  # <-- батчинг

# =========================
# Aspect ratio filtering (car-like boxes)
# =========================
ASPECT_RATIO_MIN = 1.2
ASPECT_RATIO_MAX = 3.8

def filter_detections_by_aspect_ratio(detections, rmin=1.2, rmax=3.8):
    """
    Keep only detections where ratio = max(w,h)/min(w,h) is in [rmin, rmax]
    """
    if not detections:
        return []

    out = []
    for d in detections:
        w = float(d.get("w", 0.0))
        h = float(d.get("h", 0.0))
        if w <= 0 or h <= 0:
            continue

        L = max(w, h)
        W = min(w, h)
        if W <= 0:
            continue

        ratio = L / W
        if rmin <= ratio <= rmax:
            out.append(d)

    return out

# -------------------------
# Фильтр аномальных размеров
# -------------------------

def filter_detections_by_sqrtLW_iqr(detections, k=1.5):
    """
    IQR filter on sqrt(L*W) to remove outlier boxes.
    Keeps values within [Q1-k*IQR, Q3+k*IQR].
    """
    if not detections or len(detections) < 6:
        return detections

    w = np.array([d["w"] for d in detections], dtype=np.float32)
    h = np.array([d["h"] for d in detections], dtype=np.float32)

    L = np.maximum(w, h)
    W = np.minimum(w, h)
    s = np.sqrt(L * W)

    q1 = np.percentile(s, 25)
    q3 = np.percentile(s, 75)
    iqr = q3 - q1

    lo = q1 - k * iqr
    hi = q3 + k * iqr

    out = []
    for d in detections:
        Ld = max(d["w"], d["h"])
        Wd = min(d["w"], d["h"])
        sd = math.sqrt(Ld * Wd)

        if lo <= sd <= hi:
            out.append(d)

    return out

# -------------------------
# GSD logic
# -------------------------
GSD_BASE_M_PER_PX = 0.3  # базовый масштаб датасета
GSD_TARGETS = [0.1, 0.2, 0.29, 0.4, 0.5, 0.6, 0.7]
print("GSD targets:", GSD_TARGETS)

# -------------------------
# Multi-scale inference params
# -------------------------
MIN_CARS_FOR_ANALYSIS_1 = 6
MIN_CARS_FOR_ANALYSIS_2 = 4  # <-- важно: не 1
TYPICAL_CAR_SQRTLW_M = 3.2  # sqrt(L*W) типичной машины в метрах (для первичной грубой оценки)

CHECKPOINT_EVERY_IMAGES = 20

# =========================
# Load model
# =========================
model = YOLO(MODEL_WEIGHTS)

# =========================
# Tile generator
# =========================
def generate_tiles(img_bgr: np.ndarray, tile_size: int, overlap: float):
    assert 0 <= overlap < 1
    stride = int(tile_size * (1 - overlap))
    stride = max(1, stride)

    H, W = img_bgr.shape[:2]
    xs = list(range(0, W, stride))
    ys = list(range(0, H, stride))

    for y0 in ys:
        for x0 in xs:
            x1 = min(x0 + tile_size, W)
            y1 = min(y0 + tile_size, H)

            tile = img_bgr[y0:y1, x0:x1]
            pad_right = tile_size - (x1 - x0)
            pad_bottom = tile_size - (y1 - y0)

            if pad_right > 0 or pad_bottom > 0:
                tile = cv2.copyMakeBorder(
                    tile, 0, pad_bottom, 0, pad_right,
                    borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0)
                )

            yield (x0, y0, x1, y1, tile)

# =========================
# Extract OBB xywhr
# =========================
def extract_detections_xywhr(result, keep_class_id=None):
    dets = []
    if not hasattr(result, "obb") or result.obb is None:
        return dets

    obb = result.obb
    boxes = obb.xywhr.cpu().numpy()  # (N,5): cx,cy,w,h,angle
    conf = obb.conf.cpu().numpy() if hasattr(obb, "conf") else None
    cls  = obb.cls.cpu().numpy().astype(int) if hasattr(obb, "cls") else None

    n = boxes.shape[0]
    for i in range(n):
        c = int(cls[i]) if cls is not None else -1
        if keep_class_id is not None and c != keep_class_id:
            continue

        dets.append({
            "cx": float(boxes[i, 0]),
            "cy": float(boxes[i, 1]),
            "w":  float(boxes[i, 2]),
            "h":  float(boxes[i, 3]),
            "angle": float(boxes[i, 4]),
            "conf": float(conf[i]) if conf is not None else 0.0,
            "cls": c
        })

    return dets

# =========================
# Fast merge duplicates by center hash
# =========================
def merge_xywhr_by_center_hash(detections, cell=12):
    if not detections:
        return []

    best = {}  # (gx,gy) -> det

    for d in detections:
        gx = int(d["cx"] // cell)
        gy = int(d["cy"] // cell)

        winner_key = None
        winner_conf = -1.0

        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                k = (gx + dx, gy + dy)
                if k in best and best[k]["conf"] > winner_conf:
                    winner_conf = best[k]["conf"]
                    winner_key = k

        if winner_key is None:
            best[(gx, gy)] = d
        else:
            if d["conf"] > best[winner_key]["conf"]:
                best[winner_key] = d

    return list(best.values())

# =========================
# Resize image for a target GSD
# resize_factor = GSD_BASE / GSD_TARGET
# =========================
def resize_to_gsd(img_bgr: np.ndarray, gsd_target: float, gsd_base: float = 0.3):
    resize_factor = gsd_base / gsd_target
    H, W = img_bgr.shape[:2]
    newW = max(1, int(round(W * resize_factor)))
    newH = max(1, int(round(H * resize_factor)))

    if abs(resize_factor - 1.0) < 1e-9:
        return img_bgr, 1.0

    interp = cv2.INTER_CUBIC if resize_factor > 1.0 else cv2.INTER_AREA
    img_resized = cv2.resize(img_bgr, (newW, newH), interpolation=interp)
    return img_resized, float(resize_factor)

# =========================
# Generic resize by factor (for multi-scale inference)
# =========================
def resize_by_factor(img_bgr: np.ndarray, factor: float):
    assert factor > 0
    if abs(factor - 1.0) < 1e-9:
        return img_bgr

    H, W = img_bgr.shape[:2]
    newW = max(1, int(round(W * factor)))
    newH = max(1, int(round(H * factor)))

    interp = cv2.INTER_CUBIC if factor > 1.0 else cv2.INTER_AREA
    return cv2.resize(img_bgr, (newW, newH), interpolation=interp)

# =========================
# Compute object medians + means (pixels)
# =========================
def compute_medians_means_pixels(merged):
    """
    For each detection:
      L = max(w,h)
      W = min(w,h)
      sqrtLW = sqrt(L*W)
    Returns:
      med_len, med_wid, med_sqrtLW,
      mean_len, mean_wid, mean_sqrtLW,
      count
    """
    if not merged:
        return {
            "median_length_px": np.nan,
            "median_width_px":  np.nan,
            "median_sqrtLW_px": np.nan,
            "mean_length_px": np.nan,
            "mean_width_px":  np.nan,
            "mean_sqrtLW_px": np.nan,
            "count": 0
        }

    w = np.array([d["w"] for d in merged], dtype=np.float32)
    h = np.array([d["h"] for d in merged], dtype=np.float32)

    lengths = np.maximum(w, h)
    widths  = np.minimum(w, h)
    sqrtLW  = np.sqrt(lengths * widths)

    return {
        "median_length_px": float(np.median(lengths)),
        "median_width_px":  float(np.median(widths)),
        "median_sqrtLW_px": float(np.median(sqrtLW)),
        "mean_length_px": float(np.mean(lengths)),
        "mean_width_px":  float(np.mean(widths)),
        "mean_sqrtLW_px": float(np.mean(sqrtLW)),
        "count": int(len(merged))
    }

# =========================
# Choose 3 scales based on estimated GSD
# <=0.20 -> [1.0, 0.5, 0.3]
# (0.20,0.35] -> [1.0, 0.5, 2.0]
# (0.35,0.50] -> [1.0, 0.7, 2.0]
# >0.50 -> [1.0, 1.5, 2.5]
# =========================
def choose_three_scales(gsd_est: float):
    if not np.isfinite(gsd_est) or gsd_est <= 0:
        return [1.0, 0.5, 2.0]

    if gsd_est <= 0.20:
        return [1.0, 0.5, 0.3]
    elif gsd_est <= 0.35:
        return [1.0, 0.5, 2.0]
    elif gsd_est <= 0.50:
        return [1.0, 0.7, 2.0]
    else:
        return [1.0, 1.5, 2.5]

# =========================
# Detect image tiled with BATCHING
# =========================
def detect_image_tiled_batched(img_bgr: np.ndarray, conf_thres: float, batch_size: int = 8):
    H, W = img_bgr.shape[:2]
    all_dets_global = []

    batch_tiles_rgb = []
    batch_offsets = []  # (x0,y0)

    def run_batch():
        nonlocal batch_tiles_rgb, batch_offsets, all_dets_global
        if not batch_tiles_rgb:
            return

        results = model.predict(
            source=batch_tiles_rgb,   # list of images
            imgsz=TILE_SIZE,
            conf=conf_thres,
            iou=IOU_THRES_TILE,
            device=DEVICE,
            half=HALF,
            classes=[CAR_CLASS_ID] if CAR_CLASS_ID is not None else None,
            verbose=False
        )

        # results is list aligned with input tiles
        for r0, (x0, y0) in zip(results, batch_offsets):
            dets_tile = extract_detections_xywhr(r0, keep_class_id=CAR_CLASS_ID)

            for d in dets_tile:
                cx = d["cx"] + x0
                cy = d["cy"] + y0

                if cx < 0 or cy < 0 or cx >= W or cy >= H:
                    continue

                all_dets_global.append({
                    "cx": cx, "cy": cy,
                    "w": d["w"], "h": d["h"],
                    "angle": d["angle"],
                    "conf": d["conf"],
                    "cls": d["cls"],
                })

        batch_tiles_rgb = []
        batch_offsets = []

    # collect tiles
    for x0, y0, x1, y1, tile_bgr in generate_tiles(img_bgr, TILE_SIZE, OVERLAP):
        tile_rgb = cv2.cvtColor(tile_bgr, cv2.COLOR_BGR2RGB)
        batch_tiles_rgb.append(tile_rgb)
        batch_offsets.append((x0, y0))

        if len(batch_tiles_rgb) >= batch_size:
            run_batch()

    # flush remainder
    run_batch()

    merged = merge_xywhr_by_center_hash(all_dets_global, cell=MERGE_CELL_PX)
    return merged

# =========================
# Multi-scale inference using EXISTING base detections/stats
# + aspect ratio filtering on scale2/scale3
# Outputs normalized back to base scale (divide by factor)
# =========================
def multi_scale_inference_from_base(
    img_base_bgr: np.ndarray,
    conf_used: float,
    merged_base,
    stats_base
):
    """
    base scale = 1.0 already computed outside (merged_base, stats_base)
    1) estimate gsd from median_sqrtLW_px assuming typical sqrtLW=3.2m
       gsd_est ≈ 3.2 / median_sqrtLW_px  [m/px]
    2) choose 2 additional scales
    3) run detection on scale2 & scale3
    4) normalize medians/means back to base scale by dividing by factor

    IMPORTANT:
    - scale1 stats_base are assumed to be already computed from FILTERED merged_base (done outside).
    - scale2/scale3 detections are filtered here:
        1) aspect ratio
        2) sqrtLW IQR filter
    """

    # -------------------------
    # base stats (scale 1.0)
    # -------------------------
    count1 = stats_base["count"]
    medS1 = stats_base["median_sqrtLW_px"]

    if np.isfinite(medS1) and medS1 > 0:
        gsd_est = float(TYPICAL_CAR_SQRTLW_M / medS1)
    else:
        gsd_est = np.nan

    scales = choose_three_scales(gsd_est)
    if len(scales) != 3:
        scales = [1.0, 0.5, 2.0]

    def norm(value_px, factor):
        if not np.isfinite(value_px):
            return np.nan
        return float(value_px / factor)

    def apply_filters(dets):
        """Apply filters in the correct order: aspect ratio -> IQR on sqrtLW."""
        dets = filter_detections_by_aspect_ratio(
            dets,
            rmin=ASPECT_RATIO_MIN,
            rmax=ASPECT_RATIO_MAX
        )
        dets = filter_detections_by_sqrtLW_iqr(dets, k=1.5)
        return dets

    out = {
        "conf_used": float(conf_used),
        "count_used": int(count1),
        "gsd_est_m_per_px": float(gsd_est) if np.isfinite(gsd_est) else np.nan,
        "scale_1": float(scales[0]),
        "scale_2": float(scales[1]),
        "scale_3": float(scales[2]),
    }

    # -------------------------
    # scale 1 (already computed & already filtered outside)
    # -------------------------
    out.update({
        "med_len_s1_px":     norm(stats_base["median_length_px"], scales[0]),
        "med_wid_s1_px":     norm(stats_base["median_width_px"],  scales[0]),
        "med_sqrtLW_s1_px":  norm(stats_base["median_sqrtLW_px"], scales[0]),
        "mean_len_s1_px":    norm(stats_base["mean_length_px"],   scales[0]),
        "mean_wid_s1_px":    norm(stats_base["mean_width_px"],    scales[0]),
        "mean_sqrtLW_s1_px": norm(stats_base["mean_sqrtLW_px"],   scales[0]),
        "count_s1": int(stats_base["count"])
    })

    # -------------------------
    # scale 2
    # -------------------------
    img2 = resize_by_factor(img_base_bgr, scales[1])
    merged2 = detect_image_tiled_batched(
        img2,
        conf_thres=conf_used,
        batch_size=TILE_BATCH
    )

    merged2 = apply_filters(merged2)
    st2 = compute_medians_means_pixels(merged2)

    out.update({
        "med_len_s2_px":     norm(st2["median_length_px"], scales[1]),
        "med_wid_s2_px":     norm(st2["median_width_px"],  scales[1]),
        "med_sqrtLW_s2_px":  norm(st2["median_sqrtLW_px"], scales[1]),
        "mean_len_s2_px":    norm(st2["mean_length_px"],   scales[1]),
        "mean_wid_s2_px":    norm(st2["mean_width_px"],    scales[1]),
        "mean_sqrtLW_s2_px": norm(st2["mean_sqrtLW_px"],   scales[1]),
        "count_s2": int(st2["count"])
    })

    # -------------------------
    # scale 3
    # -------------------------
    img3 = resize_by_factor(img_base_bgr, scales[2])
    merged3 = detect_image_tiled_batched(
        img3,
        conf_thres=conf_used,
        batch_size=TILE_BATCH
    )

    merged3 = apply_filters(merged3)
    st3 = compute_medians_means_pixels(merged3)

    out.update({
        "med_len_s3_px":     norm(st3["median_length_px"], scales[2]),
        "med_wid_s3_px":     norm(st3["median_width_px"],  scales[2]),
        "med_sqrtLW_s3_px":  norm(st3["median_sqrtLW_px"], scales[2]),
        "mean_len_s3_px":    norm(st3["mean_length_px"],   scales[2]),
        "mean_wid_s3_px":    norm(st3["mean_width_px"],    scales[2]),
        "mean_sqrtLW_s3_px": norm(st3["mean_sqrtLW_px"],   scales[2]),
        "count_s3": int(st3["count"])
    })

    return out

# =========================
# Collect images
# =========================
img_exts = ("*.tif",)
all_imgs = []
for e in img_exts:
    all_imgs += glob.glob(os.path.join(IMG_DIR, e))
all_imgs = sorted(all_imgs)

assert len(all_imgs) > 0, f"No images found in: {IMG_DIR}"

train_imgs, val_imgs = split_by_scene(all_imgs, val_ratio=0.15, seed=SEED)

print("Total images:", len(all_imgs))
print("Train images:", len(train_imgs))
print("Val images:", len(val_imgs))

# =========================
# Output paths (final + checkpoints)
# =========================
OUT_CSV  = "/content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels.csv"
OUT_XLSX = "/content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels.xlsx"

OUT_CSV_PART  = "/content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.csv"
OUT_XLSX_PART = "/content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.xlsx"

GSD targets: [0.1, 0.2, 0.29, 0.4, 0.5, 0.6, 0.7]
Total images: 180
Train images: 153
Val images: 27


In [9]:
# =========================
# Run TRAIN only, multi-GSD + adaptive conf + 3-scale inference
# (with aspect-ratio filtering before stats)
# MAX прогонов на один gsd:
#   Успех на 0.10: 1 (base) + 2 (scale2/scale3) = 3
#   Успех на 0.01: 1 (base 0.10) + 1 (base 0.01) + 2 = 4
#   Провал: 1 (0.10) + 1 (0.01) = 2
# =========================
total_iters = len(train_imgs) * len(GSD_TARGETS)
pbar = tqdm(total=total_iters, desc="Processing TRAIN (image x GSD)")

rows = []
processed_images = 0

for img_path in train_imgs:
    img0 = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img0 is None:
        pbar.update(len(GSD_TARGETS))
        processed_images += 1
        continue

    image_name = os.path.basename(img_path)

    for gsd_target in GSD_TARGETS:
        img_gsd, resize_factor = resize_to_gsd(
            img0,
            gsd_target=gsd_target,
            gsd_base=GSD_BASE_M_PER_PX
        )

        # -------------------------
        # PRIMARY pass (conf=0.10)
        # -------------------------
        merged_p_raw = detect_image_tiled_batched(
            img_gsd,
            conf_thres=CONF_PRIMARY,
            batch_size=TILE_BATCH
        )

        # aspect-ratio filtering BEFORE stats
        merged_p = filter_detections_by_aspect_ratio(
            merged_p_raw,
            rmin=ASPECT_RATIO_MIN,
            rmax=ASPECT_RATIO_MAX
        )
        merged_p = filter_detections_by_sqrtLW_iqr(merged_p, k=1.5)

        stats_p = compute_medians_means_pixels(merged_p)
        count_primary = stats_p["count"]

        metrics = None
        count_fallback = 0  # default

        if count_primary >= MIN_CARS_FOR_ANALYSIS_1:
            # multi-scale using already computed base (filtered detections & stats)
            metrics = multi_scale_inference_from_base(
                img_base_bgr=img_gsd,
                conf_used=CONF_PRIMARY,
                merged_base=merged_p,
                stats_base=stats_p
            )

        else:
            # -------------------------
            # FALLBACK pass (более низкий порог)
            # -------------------------
            merged_f_raw = detect_image_tiled_batched(
                img_gsd,
                conf_thres=CONF_FALLBACK,
                batch_size=TILE_BATCH
            )

            # aspect-ratio filtering BEFORE stats
            merged_f = filter_detections_by_aspect_ratio(
                merged_f_raw,
                rmin=ASPECT_RATIO_MIN,
                rmax=ASPECT_RATIO_MAX
            )
            merged_f = filter_detections_by_sqrtLW_iqr(merged_f, k=1.5)

            stats_f = compute_medians_means_pixels(merged_f)
            count_fallback = stats_f["count"]

            if count_fallback >= MIN_CARS_FOR_ANALYSIS_2:
                metrics = multi_scale_inference_from_base(
                    img_base_bgr=img_gsd,
                    conf_used=CONF_FALLBACK,
                    merged_base=merged_f,
                    stats_base=stats_f
                )
            else:
                # give up (not enough car-like detections)
                metrics = {
                    "conf_used": np.nan,
                    "count_used": int(count_fallback),
                    "gsd_est_m_per_px": np.nan,
                    "scale_1": np.nan, "scale_2": np.nan, "scale_3": np.nan,

                    "med_len_s1_px": np.nan, "med_wid_s1_px": np.nan, "med_sqrtLW_s1_px": np.nan,
                    "mean_len_s1_px": np.nan, "mean_wid_s1_px": np.nan, "mean_sqrtLW_s1_px": np.nan,
                    "count_s1": int(count_fallback),

                    "med_len_s2_px": np.nan, "med_wid_s2_px": np.nan, "med_sqrtLW_s2_px": np.nan,
                    "mean_len_s2_px": np.nan, "mean_wid_s2_px": np.nan, "mean_sqrtLW_s2_px": np.nan,
                    "count_s2": 0,

                    "med_len_s3_px": np.nan, "med_wid_s3_px": np.nan, "med_sqrtLW_s3_px": np.nan,
                    "mean_len_s3_px": np.nan, "mean_wid_s3_px": np.nan, "mean_sqrtLW_s3_px": np.nan,
                    "count_s3": 0,
                }

        rows.append({
            "image_name": image_name,
            "gsd_m_per_px": float(gsd_target),
            "resize_factor": float(resize_factor),

            # overlap + batching info (чтобы потом не гадать)
            "tile_size": int(TILE_SIZE),
            "overlap": float(OVERLAP),
            "tile_batch": int(TILE_BATCH),

            # adaptive decision info
            "conf_primary": float(CONF_PRIMARY),
            "count_primary": int(count_primary),
            "conf_fallback": float(CONF_FALLBACK),
            "count_fallback": int(count_fallback),

            # multi-scale output (or NaNs)
            **metrics
        })

        pbar.update(1)

    processed_images += 1

    # =========================
    # Checkpoint save every N images
    # =========================
    if processed_images % CHECKPOINT_EVERY_IMAGES == 0:
        df_part = pd.DataFrame(rows)
        df_part = df_part.sort_values(["image_name", "gsd_m_per_px"]).reset_index(drop=True)

        df_part.to_csv(OUT_CSV_PART, index=False)
        df_part.to_excel(OUT_XLSX_PART, index=False)

        print(f"\n[Checkpoint] Saved partial after {processed_images} images:")
        print("CSV :", OUT_CSV_PART)
        print("XLSX:", OUT_XLSX_PART)

pbar.close()

# =========================
# Final save
# =========================
df = pd.DataFrame(rows)
df = df.sort_values(["image_name", "gsd_m_per_px"]).reset_index(drop=True)

print(df.head())
print("Done. Rows:", len(df))

df.to_csv(OUT_CSV, index=False)
df.to_excel(OUT_XLSX, index=False)

print("Saved FINAL:")
print("CSV :", OUT_CSV)
print("XLSX:", OUT_XLSX)


Processing TRAIN (image x GSD):   0%|          | 0/1071 [00:21<?, ?it/s]

Processing TRAIN (image x GSD):  13%|█▎        | 140/1071 [26:05<1:47:41,  6.94s/it]


[Checkpoint] Saved partial after 20 images:
CSV : /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.csv
XLSX: /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.xlsx



Processing TRAIN (image x GSD):  26%|██▌       | 280/1071 [52:35<1:33:22,  7.08s/it]


[Checkpoint] Saved partial after 40 images:
CSV : /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.csv
XLSX: /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.xlsx



Processing TRAIN (image x GSD):  39%|███▉      | 420/1071 [1:21:32<57:12,  5.27s/it]  

...


[Checkpoint] Saved partial after 120 images:
CSV : /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.csv
XLSX: /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.xlsx



Processing TRAIN (image x GSD):  92%|█████████▏| 980/1071 [3:11:38<12:53,  8.50s/it]


[Checkpoint] Saved partial after 140 images:
CSV : /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.csv
XLSX: /content/drive/MyDrive/train_vehicle_stats_adaptive_3scale_pixels_PARTIAL.xlsx



Processing TRAIN (image x GSD): 100%|██████████| 1071/1071 [3:32:07<00:00, 11.88s/it]


    image_name  gsd_m_per_px  resize_factor  tile_size  overlap  tile_batch  \
0  austin1.tif          0.10       3.000000       1024      0.1          16   
1  austin1.tif          0.20       1.500000       1024      0.1          16   
2  austin1.tif          0.29       1.034483       1024      0.1          16   
3  austin1.tif          0.40       0.750000       1024      0.1          16   
4  austin1.tif          0.50       0.600000       1024      0.1          16   

   conf_primary  count_primary  conf_fallback  count_fallback  ...  \
0           0.1           1747           0.01               0  ...   
1           0.1           1949           0.01               0  ...   
2           0.1           1964           0.01               0  ...   
3           0.1           1525           0.01               0  ...   
4           0.1            742           0.01               0  ...   

   mean_wid_s2_px  mean_sqrtLW_s2_px  count_s2  med_len_s3_px  med_wid_s3_px  \
0       21.962753       